In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from tkinter.font import Font
from turtle import heading
from PIL import ImageTk, Image
import numpy as np
import keras
import cv2
from keras.models import load_model
import math

In [2]:
model = load_model('Model5.h5')

In [3]:
def RGB_TO_HSI(img):
    
    with np.errstate(divide='ignore', invalid='ignore'):
        
        bgr =np.float32(img)/255
        
        blue = bgr [:,:,0]
        green = bgr [:,:,1]
        red = bgr [:,:,2]
        
        def calc_intensity(red, green, blue):
            return np.divide(blue + green + red, 3)
        
        def calc_saturation(red, green, blue):
            minimum = np.minimum(np.minimum(red, green),blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)
            
            return saturation
        
        def calc_hue(red, blue, green):
            hue = np.copy(red)
            
            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue
        
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

In [7]:
#Window
main = tk.Tk()
main.title("Pendeteksi Stroberi")
main.iconbitmap("strawberry.ico")
main.configure(background="#DB7093")
main.geometry("800x600")

################################# FRAME 1 #################################
def load_frame1():
    #clear_widgets(frame1)
    frame1 = tk.Frame(main, width=800, height=600, bg="#DB7093")
    frame1.grid(row=0, column=0)  
    frame1.tkraise()
    frame1.pack_propagate(False)
       
    #Logo Widget Frame 1
    logo_img = Image.open("strawberry.png")
    logo_img = logo_img.resize((300,300))
    logo_img = ImageTk.PhotoImage(logo_img)
    logo_widget = tk.Label(frame1, image=logo_img, bg="#DB7093")
    logo_widget.image = logo_img
    logo_widget.pack(anchor=CENTER)
    
    #Instruction Widget Frame 1
    tk.Label(
        frame1,
        text="Ingin Mendeteksi Buah Stroberi?", 
        bg="#DB7093", 
        fg="black", 
        font=("TkMenuFont", 18)
    ).pack()

    #Button Widget Frame 1
    tk.Button(
        frame1,
        text="YA",
        font=("TkHeadingFont", 25),
        bg="#e54c5f",
        fg="black",
        cursor="hand2",
        activebackground="#f3d6d9",
        activeforeground="#e54c5f",
        command=lambda:load_frame2()
    ).pack(pady=20)
################################# FRAME 1 #################################

################################# FRAME 2 #################################
def load_frame2():
    #Frame Widget
    frame2 = tk.Frame(main, width=800, height=600, bg="#DB7093")
    frame2.grid(row=0, column=0)  
    frame2.pack_propagate(False)

    #Logo Widget Frame 1
    logo_img = Image.open("strawberry.png")
    logo_img = logo_img.resize((100,100))
    logo_img = ImageTk.PhotoImage(logo_img)
    logo_widget = tk.Label(frame2, image=logo_img, bg="#DB7093")
    logo_widget.image = logo_img
    logo_widget.pack()

    #Instruction Widget Frame 1
    tk.Label(
        frame2,
        text="Unggah Foto Stroberi", 
        bg="#DB7093", 
        fg="black", 
        font=("TkMenuFont", 18)
    ).pack()

    #Button Widget Frame 1
    tk.Button(
        frame2,
        text="Unggah",
        font=("TkHeadingFont", 25),
        bg="#e54c5f",
        fg="black",
        cursor="hand2",
        activebackground="#f3d6d9",
        activeforeground="#e54c5f",
        command=lambda:upload_image()
    ).pack(pady=20)
    
    #Uploaded image
    sign_image = tk.Label(frame2)
    sign_image.pack(side= TOP, expand=True)

    # Add the result text
    result_text = tk.Label(frame2, 
                           text="Hasil:", 
                           bg="#DB7093", 
                           fg="black", 
                           font=("TkMenuFont", 18))
    result_text.pack(side=BOTTOM, pady=10)

    def upload_image():
        path = filedialog.askopenfilename()
        upload = cv2.imread(path, 1)
        upload = cv2.resize(upload, (255, 255))
        hsi = RGB_TO_HSI(upload)
        mage = (hsi * 255).astype(np.uint8)
        img = cv2.cvtColor(mage, cv2.COLOR_BGR2RGB)
        im = ImageTk.PhotoImage(Image.fromarray(img))
        sign_image.configure(image=im)
        sign_image.image = im
        show_classify_button(path)
        
    def show_classify_button(path):
        classify_btn = tk.Button(frame2, 
                                 text = "Deteksi", 
                                 command=lambda: classify(path), 
                                 padx=10, 
                                 pady=5)
        classify_btn.configure(bg="#e54c5f", 
                               fg="black", 
                               cursor="hand2", 
                               activebackground="#f3d6d9", 
                               activeforeground="#e54c5f",
                               font=('TkHeadingFont', 25))
        classify_btn.place(relx=0.7, 
                           rely=0.75)


    def classify(path):
        #img = image.load_img(path, target_size = (150, 150))
        img_size=150
        img = cv2.imread(path,1)
        new_arr = cv2.resize(img, (img_size, img_size))
        x = np.array(new_arr)
        x = np.expand_dims(x, axis = 0)
        images = np.vstack([x])
        classes = model.predict(images)  
        if classes[0][0] > 0.5:
            result_text.config(text="Hasil: Mentah")
        else:
            result_text.config(text="Hasil: Matang")

        
################################# FRAME 2 #################################        


load_frame1()
#Run App
main.mainloop()

1/1 [==============================] - 0s 36ms/step
